# **Ejemplo Rápido de Evaluación**

Demostración práctica del sistema de evaluación comprehensiva.

Este notebook te muestra cómo:
1. Cargar diferentes tipos de benchmarks
2. Evaluar múltiples heurísticas
3. Analizar y comparar resultados

In [ ]:
%run librerias.ipynb
%run benchmark_loader.ipynb
%run heuristics.ipynb
%run gnn_model.ipynb

C:\Users\ismae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Librerías cargadas exitosamente
Using device: cpu
Librerías cargadas exitosamente
CARGANDO SUITE DE BENCHMARKS

Total de benchmarks cargados: 7

          nombre  nodos  aristas  densidad  grado_promedio  grado_max  grado_min  componentes  clustering  cota_superior  chi_teorico
        Petersen     10       15  0.333333        3.000000          3          3            1    0.000000              4          NaN
       Cycle_C20     20       20  0.105263        2.000000          2          2            1    0.000000              3          2.0
       Wheel_W15     15       28  0.266667        3.733333         14          3            1    0.632479             15          NaN
    Complete_K10     10       45  1.000000        9.000000          9          9            1    1.000000             10          NaN
Bipartite_K10_10     20      100  0.526316       10.000000         10         10            1    0.000000             11          2.0
        Tree_n30     30       29 

## **EJEMPLO 1: Grafo Simple Individual**

In [3]:
print("="*60)
print("EJEMPLO 1: Evaluando Grafo de Petersen")
print("="*60)

G, nombre = generar_grafo_simple('petersen')
stats = estadisticas_grafo(G, nombre)

print(f"\nGrafo: {stats['nombre']}")
print(f"Nodos: {stats['nodos']}, Aristas: {stats['aristas']}")
print(f"Densidad: {stats['densidad']:.4f}")
print(f"Grado máximo: {stats['grado_max']}")
print(f"Número cromático teórico: {stats['chi_teorico']}")
print(f"Cota superior (Δ+1): {stats['cota_superior']}")

print("\nProbando diferentes ordenamientos:")

order_random = ordenamiento_aleatorio(G)
coloring_random = greedy_coloring(G, order_random)
metrics_random = evaluar_coloracion(G, coloring_random)
print(f"  Random: {metrics_random['num_colores']} colores")

order_degree = ordenamiento_grado_desc(G)
coloring_degree = greedy_coloring(G, order_degree)
metrics_degree = evaluar_coloracion(G, coloring_degree)
print(f"  Largest Degree First: {metrics_degree['num_colores']} colores")

order_wp = ordenamiento_welsh_powell(G)
coloring_wp = greedy_coloring(G, order_wp)
metrics_wp = evaluar_coloracion(G, coloring_wp)
print(f"  Welsh-Powell: {metrics_wp['num_colores']} colores")

coloring_dsatur = dsatur_coloring(G)
metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
print(f"  DSATUR: {metrics_dsatur['num_colores']} colores")

print(f"\n✓ Mejor resultado: {min(metrics_random['num_colores'], metrics_degree['num_colores'], metrics_wp['num_colores'], metrics_dsatur['num_colores'])} colores")

EJEMPLO 1: Evaluando Grafo de Petersen

Grafo: Petersen
Nodos: 10, Aristas: 15
Densidad: 0.3333
Grado máximo: 3
Número cromático teórico: None
Cota superior (Δ+1): 4

Probando diferentes ordenamientos:
  Random: 3 colores
  Largest Degree First: 3 colores
  Welsh-Powell: 3 colores
  DSATUR: 3 colores

✓ Mejor resultado: 3 colores


## **EJEMPLO 2: Comparación en Múltiples Grafos Simples**

In [4]:
print("\n" + "="*60)
print("EJEMPLO 2: Comparación en Suite Simple")
print("="*60)

grafos_prueba = [
    generar_grafo_simple('petersen'),
    generar_grafo_simple('cycle', n=10),
    generar_grafo_simple('wheel', n=10),
    generar_grafo_simple('complete', n=5),
]

resultados_comparacion = []

for G, nombre in grafos_prueba:
    print(f"\nEvaluando: {nombre}")
    
    order_degree = ordenamiento_grado_desc(G)
    coloring_degree = greedy_coloring(G, order_degree)
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    
    coloring_dsatur = dsatur_coloring(G)
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    
    resultados_comparacion.append({
        'Grafo': nombre,
        'Nodos': G.number_of_nodes(),
        'Aristas': G.number_of_edges(),
        'LDF': metrics_degree['num_colores'],
        'DSATUR': metrics_dsatur['num_colores'],
        'Mejor': min(metrics_degree['num_colores'], metrics_dsatur['num_colores'])
    })
    
    print(f"  LDF: {metrics_degree['num_colores']} colores")
    print(f"  DSATUR: {metrics_dsatur['num_colores']} colores")

df_comparacion = pd.DataFrame(resultados_comparacion)
print("\n" + "="*60)
print("TABLA COMPARATIVA")
print("="*60)
print(df_comparacion.to_string(index=False))


EJEMPLO 2: Comparación en Suite Simple

Evaluando: Petersen
  LDF: 3 colores
  DSATUR: 3 colores

Evaluando: Cycle_C10
  LDF: 2 colores
  DSATUR: 2 colores

Evaluando: Wheel_W10
  LDF: 4 colores
  DSATUR: 4 colores

Evaluando: Complete_K5
  LDF: 5 colores
  DSATUR: 5 colores

TABLA COMPARATIVA
      Grafo  Nodos  Aristas  LDF  DSATUR  Mejor
   Petersen     10       15    3       3      3
  Cycle_C10     10       10    2       2      2
  Wheel_W10     10       18    4       4      4
Complete_K5      5       10    5       5      5


## **EJEMPLO 3: Grafo Complejo (Scale-Free)**

In [5]:
print("\n" + "="*60)
print("EJEMPLO 3: Red Scale-Free (Barabási-Albert)")
print("="*60)

G, nombre = generar_benchmark_complejo('scale_free', n=100, m=3)
stats = estadisticas_grafo(G, nombre)

print(f"\nGrafo: {stats['nombre']}")
print(f"Nodos: {stats['nodos']}, Aristas: {stats['aristas']}")
print(f"Densidad: {stats['densidad']:.4f}")
print(f"Grado promedio: {stats['grado_promedio']:.2f}")
print(f"Grado máximo: {stats['grado_max']}")
print(f"Clustering: {stats['clustering']:.4f}")

print("\nEvaluando heurísticas (puede tomar unos segundos)...")

import time

t0 = time.time()
order_degree = ordenamiento_grado_desc(G)
coloring_degree = greedy_coloring(G, order_degree)
metrics_degree = evaluar_coloracion(G, coloring_degree)
t1 = time.time()
print(f"  LDF: {metrics_degree['num_colores']} colores ({t1-t0:.4f}s)")

t0 = time.time()
order_wp = ordenamiento_welsh_powell(G)
coloring_wp = greedy_coloring(G, order_wp)
metrics_wp = evaluar_coloracion(G, coloring_wp)
t1 = time.time()
print(f"  Welsh-Powell: {metrics_wp['num_colores']} colores ({t1-t0:.4f}s)")

t0 = time.time()
coloring_dsatur = dsatur_coloring(G)
metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
t1 = time.time()
print(f"  DSATUR: {metrics_dsatur['num_colores']} colores ({t1-t0:.4f}s)")

print(f"\n✓ DSATUR típicamente da los mejores resultados en redes scale-free")


EJEMPLO 3: Red Scale-Free (Barabási-Albert)

Grafo: ScaleFree_n100_m3
Nodos: 100, Aristas: 291
Densidad: 0.0588
Grado promedio: 5.82
Grado máximo: 23
Clustering: 0.1474

Evaluando heurísticas (puede tomar unos segundos)...
  LDF: 5 colores (0.0004s)
  Welsh-Powell: 5 colores (0.0003s)
  DSATUR: 4 colores (0.0060s)

✓ DSATUR típicamente da los mejores resultados en redes scale-free


## **EJEMPLO 4: Análisis de Escalabilidad**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 4: Escalabilidad con Tamaño del Grafo")
print("="*60)

tamaños = [20, 50, 100, 200]
resultados_escalabilidad = []

for n in tamaños:
    print(f"\nGenerando grafo random con {n} nodos...")
    G, nombre = generar_grafo_simple('random', n=n, p=0.1)
    
    t0 = time.time()
    order_degree = ordenamiento_grado_desc(G)
    coloring_degree = greedy_coloring(G, order_degree)
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    tiempo_ldf = time.time() - t0
    
    t0 = time.time()
    coloring_dsatur = dsatur_coloring(G)
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    tiempo_dsatur = time.time() - t0
    
    resultados_escalabilidad.append({
        'Nodos': n,
        'Aristas': G.number_of_edges(),
        'LDF_colores': metrics_degree['num_colores'],
        'LDF_tiempo': tiempo_ldf,
        'DSATUR_colores': metrics_dsatur['num_colores'],
        'DSATUR_tiempo': tiempo_dsatur
    })
    
    print(f"  LDF: {metrics_degree['num_colores']} colores en {tiempo_ldf:.4f}s")
    print(f"  DSATUR: {metrics_dsatur['num_colores']} colores en {tiempo_dsatur:.4f}s")

df_escalabilidad = pd.DataFrame(resultados_escalabilidad)
print("\n" + "="*60)
print("ANÁLISIS DE ESCALABILIDAD")
print("="*60)
print(df_escalabilidad.to_string(index=False))


EJEMPLO 4: Escalabilidad con Tamaño del Grafo

Generando grafo random con 20 nodos...
  LDF: 3 colores en 0.0001s
  DSATUR: 3 colores en 0.0004s

Generando grafo random con 50 nodos...
  LDF: 4 colores en 0.0002s
  DSATUR: 4 colores en 0.0021s

Generando grafo random con 100 nodos...
  LDF: 6 colores en 0.0003s
  DSATUR: 6 colores en 0.0077s

Generando grafo random con 200 nodos...
  LDF: 10 colores en 0.0011s
  DSATUR: 8 colores en 0.0420s

ANÁLISIS DE ESCALABILIDAD
 Nodos  Aristas  LDF_colores  LDF_tiempo  DSATUR_colores  DSATUR_tiempo
    20       24            3    0.000091               3       0.000359
    50      108            4    0.000171               4       0.002107
   100      486            6    0.000320               6       0.007726
   200     2046           10    0.001108               8       0.042023

💡 Observación: DSATUR da mejores resultados pero es más lento en grafos grandes


## **EJEMPLO 6: Evaluación con GNN**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 6: Evaluación con GNN")
print("="*60)

print("\n1. Entrenando modelo GNN en grafo de prueba...")
G_train, nombre_train = generar_benchmark_complejo('scale_free', n=100, m=3)

# Preparar datos para GNN
def preparar_datos_gnn_local(G):
    X = extraer_features(G)
    X = normalizar_features(X)
    edge_index = np.array(list(G.edges())).T
    if edge_index.size == 0:
        edge_index = np.array([[], []], dtype=np.int64)
    edge_index_reverse = np.array([edge_index[1], edge_index[0]])
    edge_index_bidirectional = np.concatenate([edge_index, edge_index_reverse], axis=1)
    x_tensor = torch.FloatTensor(X)
    edge_index_tensor = torch.LongTensor(edge_index_bidirectional)
    from torch_geometric.data import Data
    data = Data(x=x_tensor, edge_index=edge_index_tensor, num_nodes=G.number_of_nodes())
    return data

data_train = preparar_datos_gnn_local(G_train)

# Inicializar modelo
model_gnn = GNNOrdering(in_channels=3, hidden_channels=32, num_layers=2).to(device)
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=0.01)

# Entrenar (pocas épocas para demo)
print("   Entrenando por 20 épocas...")
model_gnn.train()
for epoch in range(20):
    optimizer.zero_grad()
    scores = model_gnn(data_train.x, data_train.edge_index)
    ordering = torch.argsort(scores, descending=True).tolist()
    num_colors = greedy_coloring_gnn(data_train.edge_index, data_train.num_nodes, ordering)
    loss = torch.tensor(num_colors, dtype=torch.float32, requires_grad=True)
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
        print(f"   Época {epoch}: {num_colors} colores")

print("\n2. Evaluando GNN vs Heurísticas en grafo de prueba...")
G_test, nombre_test = generar_benchmark_complejo('scale_free', n=100, m=3)
data_test = preparar_datos_gnn_local(G_test)

# Evaluar heurísticas
t0 = time.time()
order_degree = ordenamiento_grado_desc(G_test)
coloring_degree = greedy_coloring(G_test, order_degree)
metrics_degree = evaluar_coloracion(G_test, coloring_degree)
tiempo_ldf = time.time() - t0

t0 = time.time()
coloring_dsatur = dsatur_coloring(G_test)
metrics_dsatur = evaluar_coloracion(G_test, coloring_dsatur)
tiempo_dsatur = time.time() - t0

# Evaluar GNN
model_gnn.eval()
with torch.no_grad():
    scores = model_gnn(data_test.x, data_test.edge_index)
    ordering_gnn = torch.argsort(scores, descending=True).tolist()

t0 = time.time()
colores_gnn = greedy_coloring_gnn(data_test.edge_index, data_test.num_nodes, ordering_gnn)
tiempo_gnn = time.time() - t0

print(f"\nResultados en {nombre_test}:")
print(f"  LDF:    {metrics_degree['num_colores']} colores ({tiempo_ldf:.6f}s)")
print(f"  DSATUR: {metrics_dsatur['num_colores']} colores ({tiempo_dsatur:.6f}s)")
print(f"  GNN:    {colores_gnn} colores ({tiempo_gnn:.6f}s)")

print("\n💡 Observaciones:")
print("  • GNN aprende ordenamientos específicos del grafo")
print("  • Requiere entrenamiento previo (overhead inicial)")
print("  • Puede competir con heurísticas en casos específicos")
print("  • Para mejores resultados, entrenar más épocas y con más datos")